In [8]:
import pickle
import env_methods as em
import datetime as dt
import numpy as np

In [1]:
with open("Time series result", 'rb') as f:
        miss_data = pickle.load(f)
        
with open("Nino series", 'rb') as f:
        nino_data = pickle.load(f)
        
with open("PDO series", 'rb') as f:
        PDO_data = pickle.load(f)
        
with open("AMO series", 'rb') as f:
        AMO_data = pickle.load(f)

FileNotFoundError: [Errno 2] No such file or directory: 'Time series result'

In [5]:
#Time bounds here is inclusive, exclusive, please choose them so both datasets have values in all months in that range
def pearson(climate_series, ckey, flood_series, fkey, lat, lon, start_time, end_time):
    cstart_index = em.find_closest_val(start_time, climate_series["time"])
    cend_index = em.find_closest_val(end_time, climate_series["time"])
    climate_series_data = climate_series[ckey][cstart_index:cend_index]
    #Find standard deviation of climate series
    cmean = 0
    ccount = 0
    for cdata in climate_series_data:
        if cdata != None:
            ccount += 1
            cmean += cdata
    cmean /= ccount
    
    cstdev = 0
    for cdata in climate_series_data:
        if cdata!=None:
            cstdev += (cdata - cmean) ** 2
            
    cstdev /= (ccount - 1)
    cstdev **= 0.5
    
    flat = em.find_closest_val(lat, flood_series["lat"])
    flon = em.find_closest_val(lon, flood_series["lon"])

    fstart_index = em.find_closest_val(start_time, flood_series["count"][flat][flon]["time"])
    fend_index = em.find_closest_val(end_time, flood_series["count"][flat][flon]["time"])
    #Check if this is over a river or not
    if flood_series[fkey] == None:
        return np.nan
    flood_series_data = flood_series[fkey][flat][flon][fkey][fstart_index:fend_index]
    #Find standard deviation of flood series at [lat][lon]
    fmean = 0
    fcount = 0
    for fdata in flood_series_data:
        if fdata != None:
            fcount += 1
            fmean += fdata
    fmean /= fcount
    
    fstdev = 0
    for fdata in flood_series_data:
        if fdata!=None:
            fstdev += (fdata - fmean) ** 2
    fstdev /= (fcount - 1)
    fstdev **= 0.5
    
    #Now find the covariance
    
    covar = 0
    for i in range(len(climate_series_data)):
        covar += climate_series_data[i] * flood_series_data[i]
    
    covar /= len(climate_series_data)
    
    covar -= fmean * cmean
    
    return covar/(fstdev * cstdev)

In [19]:
def pearson_mapping(climate_series, ckey, flood_series, fkey, start_time, end_time):
    result = dict()
    result["lon"] = flood_series["lon"]
    result["lat"] = flood_series["lat"]
    result["pearson"] = []
    
    for lat in flood_series["lat"]:
        t1 = dt.datetime.now()
        print("starting on latitude " + str(lat))
        row = []
        for lon in data["lon"]:
            row.append(pearson(climate_series, ckey, flood_series, fkey, lat, lon, start_time, end_time))
            
        result["pearson"].append(row)
        t2 = dt.datetime.now()
        print("Latitude " + str(lat) +" done in " + str(t2 - t1))
        
    return result